In [1]:
import pandas as pd 
import datetime as dt
import os
import pymysql
import sys
import json
import numpy_financial as np

In [2]:
fecha_inicio = dt.date(2023,1,1)
#fecha_inicio = dt.datetime.now().date()-dt.timedelta(days=120)

In [3]:
fecha_inicio.year
fecha_inicio.month
#fecha_inicio.day
str(fecha_inicio)

'2023-01-01'

In [4]:
cnx = pymysql.connect(host='159.89.90.197', port=3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')



cursor = cnx.cursor()
sql_query = """
            select id_distribuidor,id_asociado, asociado,inicio_credito,fin_credito,dispersion, capital,interes,iva,tasa,
            id_distribuidor_linea_de_credito,pago_quincenal, seguro
            from colocacion_asociado
            where date(dispersion)>='"""+str(fecha_inicio)+"""'
            and tipo_prestamo = "PRESTAMO PERSONAL RED"
            -- and id_distribuidor = '152308'
            order by dispersion asc
            """
cursor.execute(sql_query)
result = cursor.fetchall()



cursor = cnx.cursor()
sql_query_2 ="""
                select id_distribuidor, fecha_pago,capital, importe_aplicado
                from pagos_desglosados
                where date(fecha_pago)>='"""+str(fecha_inicio)+"""'
                -- and id_distribuidor = '152308'
                """

cursor.execute(sql_query_2)
result_2 = cursor.fetchall()
cnx.close()



In [5]:
results_df = pd.DataFrame(result, columns = 
                          ["id_distribuidor","id_asociado", "asociado","inicio_credito",
                           "fin_credito","dispersion","capital","interes",
                           "iva","tasa","id_distribuidor_linea_de_credito",
                          "pago_quincenal","seguro"])

results_df_2 = pd.DataFrame(result_2, columns = 
                          ["id_distribuidor","fecha_pago","capital","importe_aplicado"])

In [6]:
results_df.head()

,id_distribuidor,id_asociado,asociado,inicio_credito,fin_credito,dispersion,capital,interes,iva,tasa,id_distribuidor_linea_de_credito,pago_quincenal,seguro
0,139409,139412,ALEJANDRO ENRIQUE HERNANDEZ VILLARREAL,2023-01-09,2023-04-24,2023-01-02,14000.0,3239.13,518.26,132.0,115345,1110,464.0
1,139409,139414,MARIA DEL CARMEN LORANCA RIVERA,2023-01-09,2023-04-24,2023-01-02,10000.0,2407.35,385.18,137.0,115342,800,464.0
2,139409,139490,GLORIA ESTHER ARIDA ESCOBAR,2023-01-09,2023-04-24,2023-01-02,6000.0,1500.83,240.13,142.0,139215,484,464.0
3,139409,139491,SAYURI MICHELLE TAPIA ARIDA,2023-01-09,2023-04-24,2023-01-02,6000.0,1500.83,240.13,142.0,139211,484,464.0
4,139409,139492,QUESIA LIZBETH CRUZ LORANCA,2023-01-09,2023-04-24,2023-01-02,6000.0,1500.83,240.13,142.0,139351,484,464.0


In [7]:
results_df_2

,id_distribuidor,fecha_pago,capital,importe_aplicado
0,111373,2023-10-01 00:00:00,503.1,670
1,155622,2023-10-01 00:00:00,6000,6000
2,16661,2023-10-01 00:00:00,487.12,1013
3,77889,2023-10-01 00:00:00,0,11
4,49302,2023-10-01 00:00:00,1810.58,2798
...,...,...,...,...
81101,149358,2024-05-15 18:16:04,593.27,1355
81102,181548,2024-05-15 19:30:57,8290.66,10000
81103,181548,2024-05-15 19:33:09,962,1184
81104,74349,2024-05-15 19:44:04,11384.66,20000


In [8]:
payments_dict = dict()

In [9]:
for index,row in results_df_2.iterrows():
    if row["id_distribuidor"] not in payments_dict:
        payments_dict[row["id_distribuidor"]]=[[],[],[]]
    
    payments_dict[row["id_distribuidor"]][0].append(row["fecha_pago"].date())
    payments_dict[row["id_distribuidor"]][1].append(row["capital"])
    payments_dict[row["id_distribuidor"]][2].append(row["importe_aplicado"])

In [10]:
payments_dict

{'111373': [[datetime.date(2023, 10, 1),
   datetime.date(2023, 1, 20),
   datetime.date(2023, 8, 14),
   datetime.date(2023, 8, 30),
   datetime.date(2023, 9, 16),
   datetime.date(2023, 10, 1),
   datetime.date(2023, 10, 6),
   datetime.date(2023, 10, 17),
   datetime.date(2023, 11, 1),
   datetime.date(2023, 11, 7)],
  ['503.1',
   '92.84',
   '390.75',
   '429.06',
   '471.13',
   '503.1',
   '14.21',
   '568.03',
   '607.5',
   '16.21'],
  ['670', '158.82', '746', '750', '750', '670', '87', '743', '668', '81.21']],
 '155622': [[datetime.date(2023, 10, 1),
   datetime.date(2023, 10, 1),
   datetime.date(2023, 10, 1),
   datetime.date(2023, 10, 1),
   datetime.date(2023, 10, 1),
   datetime.date(2023, 10, 1),
   datetime.date(2023, 6, 29),
   datetime.date(2023, 6, 29),
   datetime.date(2023, 6, 29),
   datetime.date(2023, 6, 29),
   datetime.date(2023, 7, 6),
   datetime.date(2023, 7, 6),
   datetime.date(2023, 7, 6),
   datetime.date(2023, 7, 10),
   datetime.date(2023, 7, 13),
  

In [11]:
new_dict = {"id_asociado":list(), "asociado":list(),"fecha_amortizacion":list(),
            "pago_total":list(),"capital":list(),"saldo":list(),
            "interes":list(),"iva":list(),"tasa":list(),"id_distribuidor":list(),
            #"capital_pago":list()
            "pago_acumulado":list(),"pago_esperado_acumulado":list(), "fecha_pago":list(),
            "%_pagado":list()
           }

pagos_esperados_dict = dict()

for index, row in results_df.iterrows():
    if row["id_distribuidor"] not in pagos_esperados_dict:
            pagos_esperados_dict[row["id_distribuidor"]]=0.0
    pagos_esperados_dict[row["id_distribuidor"]]+=float(row["pago_quincenal"]) + (float(row["seguro"])/16.00)

for index, row in results_df.iterrows():
    iterative_date = row["inicio_credito"]
    tasa = (row["tasa"]/52)/100
    semanas = 16
    saldo = row["capital"]
    pct_iva = 0.16  
    #amortizacion_counter=0
    cummulative_payment=0.0
    cummulative_expected=0.0
    for i in range(16):
        new_dict["id_distribuidor"].append(row["id_distribuidor"])
        new_dict["id_asociado"].append(row["id_asociado"])
        new_dict["asociado"].append(row["asociado"])
        fecha_amortizacion = iterative_date
        new_dict["fecha_amortizacion"].append(fecha_amortizacion)
        iterative_date = iterative_date + dt.timedelta(days=7)
        
        new_dict["saldo"].append(saldo)
        
        capital = np.ppmt(tasa,i+1,semanas,-row["capital"])
        new_dict["capital"].append(capital)
        
        interes = np.ipmt(tasa,i+1,semanas,-saldo)
        new_dict["interes"].append(interes)
        
        new_dict["iva"].append(interes*pct_iva)
        pago_total = capital+interes + (interes*pct_iva)
        new_dict["pago_total"].append(pago_total)
        saldo -= capital
        
        new_dict["tasa"].append(row["tasa"])
        
        cummulative_expected+=pagos_esperados_dict[row["id_distribuidor"]]
        #cummulative_payment=0.0
        if row["id_distribuidor"] in payments_dict:
            date_counter=0
            fecha_pago = ""
            
            for date in payments_dict[row["id_distribuidor"]][0]:
                fecha_final=fecha_amortizacion+dt.timedelta(days=6)
                if date_counter==0:
                    fecha_amortizacion-=dt.timedelta(days=6)
                if fecha_amortizacion <= date <= fecha_amortizacion+dt.timedelta(days=6):
                    payment_amount=float(payments_dict[row["id_distribuidor"]][2][date_counter])
                    cummulative_payment+=payment_amount
                    fecha_pago = date
                    #print(payment_amount,date)
                date_counter+=1
        new_dict["fecha_pago"].append(fecha_pago)
        #new_dict["capital_pago"].append(payment_amount)
        new_dict["pago_acumulado"].append(cummulative_payment)
        new_dict["pago_esperado_acumulado"].append(cummulative_expected)
        new_dict["%_pagado"].append(cummulative_payment/cummulative_expected)
        #amortizacion_counter+=1
    

In [12]:
processed_df = pd.DataFrame.from_dict(new_dict)

In [13]:
processed_df.head()

,id_asociado,asociado,fecha_amortizacion,pago_total,capital,saldo,interes,iva,tasa,id_distribuidor,pago_acumulado,pago_esperado_acumulado,fecha_pago,%_pagado
0,139412,ALEJANDRO ENRIQUE HERNANDEZ VILLARREAL,2023-01-09,1132.459760,720.213606,14000.000000,355.38461538461536,56.861538,132.0,139409,3507.0,3507.0,2023-01-06,1.00
1,139412,ALEJANDRO ENRIQUE HERNANDEZ VILLARREAL,2023-01-16,1109.418060,738.495952,13279.786394,319.7604384560711,51.161670,132.0,139409,7014.0,7014.0,2023-01-13,1.00
2,139412,ALEJANDRO ENRIQUE HERNANDEZ VILLARREAL,2023-01-23,1088.057234,757.242387,12541.290442,285.1852129924639,45.629634,132.0,139409,10521.0,10521.0,2023-01-20,1.00
3,139412,ALEJANDRO ENRIQUE HERNANDEZ VILLARREAL,2023-01-30,1068.536443,776.464694,11784.048055,251.7859902413082,40.285758,132.0,139409,10521.0,14028.0,,0.75
4,139412,ALEJANDRO ENRIQUE HERNANDEZ VILLARREAL,2023-02-06,1051.024875,796.174952,11007.583361,219.6982097584143,35.151714,132.0,139409,17535.0,17535.0,2023-02-03,1.00


In [14]:
processed_df.query("id_asociado=='155457'")

,id_asociado,asociado,fecha_amortizacion,pago_total,capital,saldo,interes,iva,tasa,id_distribuidor,pago_acumulado,pago_esperado_acumulado,fecha_pago,%_pagado
74912,155457,NIDIA ALEJANDRA MORALES AMADO,2023-06-27,561.141573,362.826471,6999.990000,170.96129423076925,27.353807,127.0,155456,3885.0,3885.0,2023-06-27,1.000000
74913,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-04,549.977399,371.687810,6637.163529,153.69792128706777,24.591667,127.0,155456,7770.0,7770.0,2023-07-04,1.000000
74914,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-11,539.645504,380.765570,6265.475718,136.96545992610248,21.914474,127.0,155456,11655.0,11655.0,2023-07-11,1.000000
74915,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-18,530.220845,390.065037,5884.710148,120.82397255059543,19.331836,127.0,155456,15540.0,15540.0,2023-07-18,1.000000
74916,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-25,521.782913,399.591625,5494.645111,105.33731701801503,16.853971,127.0,155456,19425.0,19425.0,2023-07-25,1.000000
74917,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-01,514.415974,409.350882,5095.053486,90.57335446672526,14.491737,127.0,155456,23310.0,23310.0,2023-08-01,1.000000
74918,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-08,508.209325,419.348491,4685.702603,76.60416791019378,12.256667,127.0,155456,27195.0,27195.0,2023-08-08,1.000000
74919,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-15,503.257570,429.590271,4266.354113,63.50629214369074,10.161007,127.0,155456,27195.0,31080.0,,0.875000
74920,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-22,499.660896,440.082187,3836.763842,51.360955535121434,8.217753,127.0,155456,34965.0,34965.0,2023-08-22,1.000000
74921,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-29,497.525376,450.830348,3396.681655,40.25433430020167,6.440693,127.0,155456,34965.0,38850.0,,0.900000


In [15]:
max_pago_acumulado = processed_df.groupby("id_asociado")["pago_acumulado"].max()
max_pago_esperado = processed_df.groupby("id_asociado")["pago_esperado_acumulado"].max()
max_pago_acumulado_df = max_pago_acumulado.reset_index()
max_pago_esperado_df = max_pago_esperado.reset_index()


In [16]:
max_pago_acumulado_df.query("id_asociado=='155457'")


,id_asociado,pago_acumulado
4629,155457,62160.0


In [17]:
max_pago_esperado_df.query("id_asociado=='155457'")

,id_asociado,pago_esperado_acumulado
4629,155457,62160.0


In [18]:
for index, row in processed_df.iterrows():
    if  not row['fecha_pago']:
        #print(row["pago_acumulado"])
        #print(max_pago_acumulado_df[max_pago_acumulado_df['id_asociado'] == row["id_asociado"]]['pago_acumulado'].iloc[0])
        if row["pago_acumulado"]<= max_pago_acumulado_df[max_pago_acumulado_df['id_asociado'] == row["id_asociado"]]['pago_acumulado'].iloc[0]:
            processed_df.at[index, 'fecha_pago'] = row["fecha_amortizacion"]
            #print(row["fecha_amortizacion"])

In [19]:
processed_df.query("id_asociado=='155457'")

,id_asociado,asociado,fecha_amortizacion,pago_total,capital,saldo,interes,iva,tasa,id_distribuidor,pago_acumulado,pago_esperado_acumulado,fecha_pago,%_pagado
74912,155457,NIDIA ALEJANDRA MORALES AMADO,2023-06-27,561.141573,362.826471,6999.990000,170.96129423076925,27.353807,127.0,155456,3885.0,3885.0,2023-06-27,1.000000
74913,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-04,549.977399,371.687810,6637.163529,153.69792128706777,24.591667,127.0,155456,7770.0,7770.0,2023-07-04,1.000000
74914,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-11,539.645504,380.765570,6265.475718,136.96545992610248,21.914474,127.0,155456,11655.0,11655.0,2023-07-11,1.000000
74915,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-18,530.220845,390.065037,5884.710148,120.82397255059543,19.331836,127.0,155456,15540.0,15540.0,2023-07-18,1.000000
74916,155457,NIDIA ALEJANDRA MORALES AMADO,2023-07-25,521.782913,399.591625,5494.645111,105.33731701801503,16.853971,127.0,155456,19425.0,19425.0,2023-07-25,1.000000
74917,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-01,514.415974,409.350882,5095.053486,90.57335446672526,14.491737,127.0,155456,23310.0,23310.0,2023-08-01,1.000000
74918,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-08,508.209325,419.348491,4685.702603,76.60416791019378,12.256667,127.0,155456,27195.0,27195.0,2023-08-08,1.000000
74919,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-15,503.257570,429.590271,4266.354113,63.50629214369074,10.161007,127.0,155456,27195.0,31080.0,2023-08-15,0.875000
74920,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-22,499.660896,440.082187,3836.763842,51.360955535121434,8.217753,127.0,155456,34965.0,34965.0,2023-08-22,1.000000
74921,155457,NIDIA ALEJANDRA MORALES AMADO,2023-08-29,497.525376,450.830348,3396.681655,40.25433430020167,6.440693,127.0,155456,34965.0,38850.0,2023-08-29,0.900000


In [20]:
processed_df.to_excel("./data/amortizaciones_"+str(fecha_inicio)+"_"+str(dt.datetime.now().date())+".xlsx")  
